In [1]:
import sys
sys.path.insert(0, './algorithms/alpha_beta')
sys.path.insert(0, './algorithms/dqn')
sys.path.insert(0, './algorithms/stockfish')
import numpy as np

In [2]:
from algorithms.MyChessBoard import MyChessBoard

from algorithms.alpha_beta.Alpha_Beta import find_move as alpha_beta_find_move

from algorithms.dqn.dqn_65.bot_65 import find_move as dqn_65_find_move
from algorithms.dqn.dqn_65.model65 import get_model65

from algorithms.dqn.dqn_888.bot_888 import find_move as dqn_888_find_move
from algorithms.dqn.dqn_888.model888 import get_model888

from algorithms.stockfish.stockfish_bot import find_move as stockfish_find_move


2022-01-18 00:32:21.339110: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-18 00:32:21.339145: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
board = MyChessBoard()

In [4]:
len(list(board.legal_moves()))

20

In [5]:
model65 = get_model65(STATE_SHAPE= (65, ), NB_ACTIONS= 4096)
model888 = get_model888(STATE_SHAPE= (8, 8, 8), NB_ACTIONS= 4096)


2022-01-18 00:32:23.431992: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-18 00:32:23.432071: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-18 00:32:23.432144: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LoG): /proc/driver/nvidia/version does not exist
2022-01-18 00:32:23.432729: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
board.convert_board_to_int()

array([[ -40.,  -20.,  -30.,  -50., -900.,  -30.,  -20.,  -40.],
       [ -10.,  -10.,  -10.,  -10.,  -10.,  -10.,  -10.,  -10.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [  10.,   10.,   10.,   10.,   10.,   10.,   10.,   10.],
       [  40.,   20.,   30.,   50.,  900.,   30.,   20.,   40.]])

In [7]:
state = board.get_state()
dqn_65_find_move(board, model65)

2022-01-18 00:32:25.291986: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-18 00:32:25.310198: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 1999965000 Hz


Move.from_uci('g2g3')

In [8]:
name = ["Alpha_Beta", "DQN 65", "dqn 888", "Stockfish"]
find = [alpha_beta_find_move, dqn_65_find_move, dqn_888_find_move, stockfish_find_move]

result = np.zeros((4, 4))
mean_step = np.zeros((4, 4))

In [9]:
num_battle_with_com1 = 1
com1 = 0

com2 = 2
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    step = 0
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board, model888))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board, model888))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)

0 vs 2 : 0
Số quân: <bound method MyChessBoard.count_pieces of <algorithms.MyChessBoard.MyChessBoard object at 0x7fb13f98d7c0>>, độ sâu: 3
Số quân: <bound method MyChessBoard.count_pieces of <algorithms.MyChessBoard.MyChessBoard object at 0x7fb13f98d7c0>>, độ sâu: 3
Số quân: <bound method MyChessBoard.count_pieces of <algorithms.MyChessBoard.MyChessBoard object at 0x7fb13f98d7c0>>, độ sâu: 3
Số quân: <bound method MyChessBoard.count_pieces of <algorithms.MyChessBoard.MyChessBoard object at 0x7fb13f98d7c0>>, độ sâu: 3
Số quân: <bound method MyChessBoard.count_pieces of <algorithms.MyChessBoard.MyChessBoard object at 0x7fb13f98d7c0>>, độ sâu: 3
Số quân: <bound method MyChessBoard.count_pieces of <algorithms.MyChessBoard.MyChessBoard object at 0x7fb13f98d7c0>>, độ sâu: 3
Số quân: <bound method MyChessBoard.count_pieces of <algorithms.MyChessBoard.MyChessBoard object at 0x7fb13f98d7c0>>, độ sâu: 3
Số quân: <bound method MyChessBoard.count_pieces of <algorithms.MyChessBoard.MyChessBoard obj

In [10]:
result

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [11]:
num_step

[33]

In [ ]:
num_battle_with_com1 = 50
com1 = 0

com2 = 1
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    step = 0
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board, model_dqn))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board, model_dqn))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)
mean_step[com1, com2] = np.mean(num_step)


com2 = 2
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)
mean_step[com1, com2] = np.mean(num_step)


com2 = 3
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)
mean_step[com1, com2] = np.mean(num_step)


0 vs 1 : 0
Số quân: <bound method MyChessBoard.count_pieces of <algorithms.MyChessBoard.MyChessBoard object at 0x7f82501c4cd0>>, độ sâu: 3


NameError: name 'model_dqn' is not defined

In [ ]:
com1 = 1
num_battle_with_com1 = 100

com2 = 2
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board, model_dqn))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board, model_dqn))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)
mean_step[com1, com2] = np.mean(num_step)

com2 = 3
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board, model_dqn))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board, model_dqn))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)
mean_step[com1, com2] = np.mean(num_step)

1 vs 2 : 0
fivefold repetition
1 vs 2 : 1
fivefold repetition
1 vs 2 : 2
fivefold repetition
1 vs 2 : 3
fivefold repetition
1 vs 2 : 4
fivefold repetition
1 vs 2 : 5
fivefold repetition
1 vs 2 : 6
fivefold repetition
1 vs 2 : 7
fivefold repetition
1 vs 2 : 8
fivefold repetition
1 vs 2 : 9
fivefold repetition
1 vs 2 : 10
fivefold repetition
1 vs 2 : 11
fivefold repetition
1 vs 2 : 12
fivefold repetition
1 vs 2 : 13
fivefold repetition
1 vs 2 : 14
fivefold repetition
1 vs 2 : 15
fivefold repetition
1 vs 2 : 16
fivefold repetition
1 vs 2 : 17
fivefold repetition
1 vs 2 : 18
fivefold repetition
1 vs 2 : 19
fivefold repetition
1 vs 2 : 20
fivefold repetition
1 vs 2 : 21
fivefold repetition
1 vs 2 : 22
fivefold repetition
1 vs 2 : 23
fivefold repetition
1 vs 2 : 24
fivefold repetition
1 vs 2 : 25
fivefold repetition
1 vs 2 : 26
fivefold repetition
1 vs 2 : 27
fivefold repetition
1 vs 2 : 28
fivefold repetition
1 vs 2 : 29
fivefold repetition
1 vs 2 : 30
fivefold repetition
1 vs 2 : 31
fivefo

In [ ]:
com1 = 2
num_battle_with_com1 = 100

com2 = 3
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)
mean_step[com1, com2] = np.mean(num_step)


2 vs 3 : 0
2 vs 3 : 1
2 vs 3 : 2
2 vs 3 : 3
2 vs 3 : 4
2 vs 3 : 5
2 vs 3 : 6
2 vs 3 : 7
2 vs 3 : 8
2 vs 3 : 9
2 vs 3 : 10
2 vs 3 : 11
2 vs 3 : 12
2 vs 3 : 13
2 vs 3 : 14
2 vs 3 : 15
2 vs 3 : 16
2 vs 3 : 17
2 vs 3 : 18
2 vs 3 : 19
2 vs 3 : 20
2 vs 3 : 21
2 vs 3 : 22
2 vs 3 : 23
2 vs 3 : 24
2 vs 3 : 25
2 vs 3 : 26
2 vs 3 : 27
2 vs 3 : 28
2 vs 3 : 29
2 vs 3 : 30
2 vs 3 : 31
2 vs 3 : 32
2 vs 3 : 33
2 vs 3 : 34
2 vs 3 : 35
2 vs 3 : 36
2 vs 3 : 37
2 vs 3 : 38
2 vs 3 : 39
2 vs 3 : 40
2 vs 3 : 41
2 vs 3 : 42
2 vs 3 : 43
2 vs 3 : 44
2 vs 3 : 45
2 vs 3 : 46
2 vs 3 : 47
2 vs 3 : 48
2 vs 3 : 49
2 vs 3 : 50
2 vs 3 : 51
2 vs 3 : 52
2 vs 3 : 53
2 vs 3 : 54
2 vs 3 : 55
2 vs 3 : 56
2 vs 3 : 57
2 vs 3 : 58
2 vs 3 : 59
2 vs 3 : 60
2 vs 3 : 61
2 vs 3 : 62
2 vs 3 : 63
2 vs 3 : 64
2 vs 3 : 65
2 vs 3 : 66
2 vs 3 : 67
2 vs 3 : 68
2 vs 3 : 69
2 vs 3 : 70
2 vs 3 : 71
2 vs 3 : 72
2 vs 3 : 73
2 vs 3 : 74
2 vs 3 : 75
2 vs 3 : 76
2 vs 3 : 77
2 vs 3 : 78
2 vs 3 : 79
2 vs 3 : 80
2 vs 3 : 81
2 vs 3 : 82
2 vs 3 : 83
2 

In [ ]:
result

array([[  0.,  50.,  50.,   0.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [ 50., 100., 100.,   0.]])

In [ ]:
mean_steps

array([[   0.  ,   20.5 ,  395.  , 1401.34],
       [   0.  ,    0.  , 2823.25, 4121.2 ],
       [   0.  ,    0.  ,    0.  , 5456.11],
       [   0.  ,    0.  ,    0.  ,    0.  ]])